<a href="https://colab.research.google.com/github/zstunsta/odc-colab/blob/master/L8NDVI_NDBI_NDWI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Landsat-8 NDVI NDBI NDWI Calculation Using Google Earth Engine and Open Data Cube
Ls8 Reference Info: https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri/

Source Code: https://colab.research.google.com/github/ceos-seo/odc-colab/blob/master/notebooks/01.01.Getting_Started_ODC_and_Colab.ipynb#scrollTo=RGFU03tN4h8k

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
!wget -nc https://raw.githubusercontent.com/ceos-seo/odc-colab/master/odc_colab.py
from odc_colab import odc_colab_init
odc_colab_init(install_odc_gee=True)

--2021-10-07 20:48:21--  https://raw.githubusercontent.com/ceos-seo/odc-colab/master/odc_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15292 (15K) [text/plain]
Saving to: ‘odc_colab.py’

odc_colab.py        100%[===================>]  14.93K  --.-KB/s    in 0s      

2021-10-07 20:48:22 (110 MB/s) - ‘odc_colab.py’ saved [15292/15292]

Module utils was not found; cloning https://github.com/ceos-seo/data_cube_utilities.git to CWD...
Package postgresql was not found; installing it...
Module odc-gee was not found; cloning https://github.com/ceos-seo/odc-gee.git to CWD...


In [74]:
from odc_colab import populate_db
populate_db()

No database file supplied. Downloading default index.
Lockfile exists, skipping population.


In [73]:
# Suppress Warning Messages
import warnings
warnings.filterwarnings('ignore')

# Load Data Cube Configuration
from odc_gee import earthengine
dc = earthengine.Datacube(app='Getting_Started_loading_data')

# Import Data Cube API
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi()

# Import Utilities
import xarray as xr

Within this ODC instance, 'products' refer to satellites (platform), and their instruments. To see what products are available, you can run the following command.

In [75]:
products = dc.list_products()

display_columns = ["name",
                   "description",
                   "platform",
                   "instrument",
                   "crs",
                   "resolution"]

products[display_columns].sort_index()

,name,description,platform,instrument,crs,resolution
id,,,,,,
1,ls8_google,<p>This dataset is the atmospherically correct...,LANDSAT_8,OLI/TIRS,EPSG:4326,"(-0.000269493352, 0.000269493352)"
2,s1_google,<p>The Sentinel-1 mission provides data from a...,Sentinel-1A,C-SAR,EPSG:4326,"(-8.98311175e-05, 8.98311175e-05)"
3,s2_google,"<p>Sentinel-2 is a wide-swath, high-resolution...",Sentinel-2A,MSI,EPSG:4326,"(-8.98311175e-05, 8.98311175e-05)"
4,ls7_google,<p>This dataset is the atmospherically\ncorrec...,LANDSAT_7,ETM,EPSG:4326,"(-0.000269493352, 0.000269493352)"
5,viirs_google,<p>Monthly average radiance composite images u...,VIIRS,DNB,EPSG:4326,"(-0.00416666667, 0.00416666667)"
6,palsar_google,<p>The global 25m PALSAR/PALSAR-2 mosaic is a ...,ALOS,SAR,EPSG:4326,"(-0.000224577794, 0.000224577794)"
7,proba_google,<p>The Copernicus Global Land Service (CGLS) i...,PROBA-V,CGLS-LC,EPSG:4326,"(-0.000898311175, 0.000898311175)"


In [71]:
# Load Data
product = "ls8_google"
platform = "LANDSAT_8"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
B1,B1,int16,,-9999.0,"[ultra_blue, band_1, b1]",NaN
B2,B2,int16,,-9999.0,"[blue, band_2, b2]",NaN
B3,B3,int16,,-9999.0,"[green, band_3, b3]",NaN
B4,B4,int16,,-9999.0,"[red, band_4, b4]",NaN
B5,B5,int16,,-9999.0,"[nir, band_5, b5]",NaN
B6,B6,int16,,-9999.0,"[swir1, band_6, b6]",NaN
B7,B7,int16,,-9999.0,"[swir2, band_7, b7]",NaN
B10,B10,int16,Kelvin,-9999.0,"[tirs1, band_10_brightness_temperature, b10]",NaN
B11,B11,int16,Kelvin,-9999.0,"[tirs2, band_11_brightness_temperature, b11]",NaN


In [52]:
from utils.data_cube_utilities.dc_display_map import display_map
# Load the plotting utility
from utils.data_cube_utilities.dc_rgb import rgb
import matplotlib.pyplot as plt

In [72]:

# Define Time Range - Select a time period within the extents of the dataset (Year-Month-Day)
# Landsat-8 time range: 07-Apr-2013 to current
time_extents = ('2020-01-01', '2020-12-31')

# Specify box centre and box size in degrees.
# Vicuña, Chile
lat_long = (-30, -70.75)
box_size_deg = 0.25

# La Serena, Chile
#lat_long = (-29.90, -71.20)
#box_size_deg = 0.25

latitude = (lat_long[0]-box_size_deg/2, lat_long[0]+box_size_deg/2)
longitude = (lat_long[1]-box_size_deg/2, lat_long[1]+box_size_deg/2)
print(measurements)

#display_map(latitude,longitude)

                             name  ...                                   flags_definition
product      measurement           ...                                                   
ls7_google   B1                B1  ...                                                NaN
             B2                B2  ...                                                NaN
             B3                B3  ...                                                NaN
             B4                B4  ...                                                NaN
             B5                B5  ...                                                NaN
...                           ...  ...                                                ...
s2_google    TCI_G          TCI_G  ...                                                NaN
             TCI_B          TCI_B  ...                                                NaN
             QA60            QA60  ...  {'cirrus_clouds': {'bits': [11], 'values': {'0...
viirs_goog

In [64]:
print(time_extents)
print(dc)

('2020-01-01', '2020-12-31')
Datacube<index=Index<db=PostgresDb<engine=Engine(postgresql://root@:5432/datacube)>>>


In [67]:
# Load data
ds = dc.load(product=product,
             x=latitude,
             y=longitude,
             time=time_extents,
             measurements=['red', 'green', 'blue', 'nir', 'swir1', 'swir2'])

print(ds)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*


In [60]:
# Select one of the time slices and create an output image. 
# Clouds will be visible in WHITE for an output image

slice = 10  # select the time slice number here

# Select the output image bands
# Users can create other combinations of bands (loaded above), as desired
# True-Color = red, green, blue (this is the common true-color RGB image)
# False Color = swir2, nir, green (this is commonly used for Landsat data viewing)

true_rgb = ds.isel(time=slice)[['red', 'green', 'blue']].to_array()
false_rgb = ds.isel(time=slice)[['swir2', 'nir', 'green']].to_array()

# Define the plot settings and show the plots
# Users may want to alter the figure sizes or plot titles
# The "vmax" value controls the brightness of the images and can be adjusted 

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
true_rgb.plot.imshow(ax=ax[0], vmin=0, vmax=3000)
false_rgb.plot.imshow(ax=ax[1], vmin=0, vmax=5000)
ax[0].set_title('True Color'), ax[0].xaxis.set_visible(False), ax[0].yaxis.set_visible(False)
ax[1].set_title('False Color'), ax[1].xaxis.set_visible(False), ax[1].yaxis.set_visible(False)
plt.show()

ValueError: ignored